# Hitansh Agarwal

## Reg No.: 230957218

## Roll No.: 73

### Week 5

In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("weatherAUS.csv")
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [5]:
columns_to_drop = ['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RISK_MM']
df = df.drop(columns=columns_to_drop)
df = df.dropna()

In [8]:
X = df.drop('RainTomorrow', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, df['RainTomorrow'], test_size=0.2, shuffle=True)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy before VIF computation: {accuracy:.2f}')

Accuracy before VIF computation: 0.85


In [9]:
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

,Feature,VIF
0,MinTemp,57.816330
1,MaxTemp,608.073689
2,Rainfall,1.274984
3,Evaporation,7.093691
4,Sunshine,17.288336
5,WindGustSpeed,26.320937
6,WindSpeed9am,7.776568
7,WindSpeed3pm,13.476810
8,Humidity9am,60.741577
9,Humidity3pm,47.893909


In [10]:
high_vif_features = vif_data[vif_data['VIF'] > 5]['Feature'].tolist()
print(f"Hight VIF features: {high_vif_features}")

if 'Humidity9am' in high_vif_features and 'Humidity3pm' in high_vif_features:
    df['Humidity_Diff'] = df['Humidity3pm'] - df['Humidity9am']


df = df.drop(columns=high_vif_features)

Hight VIF features: ['MinTemp', 'MaxTemp', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']


In [11]:
X = df.drop('RainTomorrow', axis=1)
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

,Feature,VIF
0,Rainfall,1.033151
1,Humidity_Diff,1.033151


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, df['RainTomorrow'], test_size=0.2, shuffle=True)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy:.2f}')

Accuracy of the model: 0.81
